# Cleaning a PostgreSQL Database
![Clean PostgreSQL Database](Project_Image.jpeg)

In this project, you will work with data from a hypothetical Super Store to challenge and enhance your SQL skills in data cleaning. This project will engage you in identifying top categories based on the highest profit margins and detecting missing values, utilizing your comprehensive knowledge of SQL concepts.

## Data Dictionary:

### `orders`:
| Column | Definition | Data type | Comments |
|--------|------------|-----------|----------|
| `row_id`| Unique Record ID | `INTEGER` |
| `order_id` | Identifier for each order in table | `TEXT` | Connects to `order_id` in `returned_orders` table |
| `order_date` | Date when order was placed | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |
| `region` | Region Customer belongs to | `TEXT` | Connects to `region` in `people` table |
| `product_id` | Identifier of Product bought | `TEXT` | Connects to `product_id` in `products` table |
| `sales` | Total Sales Amount for the Line Item | `DOUBLE PRECISION` |
| `quantity` | Total Quantity for the Line Item | `DOUBLE PRECISION` |
| `discount` | Discount applied for the Line Item | `DOUBLE PRECISION` |
| `profit` | Total Profit earned on the Line Item | `DOUBLE PRECISION` |

### `returned_orders`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `returned`| Yes values for Order / Line Item Returned | `TEXT` |
| `order_id` | Identifier for each order in table | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |

### `people`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `person`| Name of Salesperson credited with Order | `TEXT` |
| `region` | Region Salesperson in operating in | `TEXT` |

### `products`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `product_id`| Unique Identifier for the Product | `TEXT` |
| `category` | Category Product belongs to | `TEXT` |
| `sub_category` | Sub Category Product belongs to | `TEXT` |
| `product_name` | Detailed Name of the Product | `TEXT` |

As you can see in the Data Dictionary above, date fields have been written to the `orders` table as `TEXT` and numeric fields like sales, profit, etc. have been written to the `orders` table as `Double Precision`. You will need to take care of these types in some of the queries. This project is an excellent opportunity to apply your SQL skills in a practical setting and gain valuable experience in data cleaning and analysis. Good luck, and happy querying!

In [1]:
-- top_five_products_each_category
WITH cte AS (
	SELECT 
	p.category,
	p.product_name,
	ROUND(SUM(o.sales) :: numeric,2) AS product_total_sales,
	ROUND(SUM(o.profit) :: numeric,2) AS product_total_profit
		
	FROM public.orders AS o
	LEFT JOIN public.products AS p
	USING(product_id)
	GROUP BY p.category,p.product_name
),
ranked AS (
	SELECT * ,
	 ROW_NUMBER() OVER(PARTITION BY category ORDER BY product_total_sales DESC ) AS product_rank
	FROM cte 
)
SELECT * 
FROM ranked 
WHERE product_rank BETWEEN 1 AND 5
ORDER BY category


category  ... product_rank
0         Furniture  ...            1
1         Furniture  ...            2
2         Furniture  ...            3
3         Furniture  ...            4
4         Furniture  ...            5
5   Office Supplies  ...            1
6   Office Supplies  ...            2
7   Office Supplies  ...            3
8   Office Supplies  ...            4
9   Office Supplies  ...            5
10       Technology  ...            1
11       Technology  ...            2
12       Technology  ...            3
13       Technology  ...            4
14       Technology  ...            5

[15 rows x 5 columns]

In [2]:
-- impute_missing_values
WITH unit_prices AS (
	SELECT 
		market,
		region,
		product_id,
		AVG(sales/(quantity*(1-discount))) AS unit_price 
	FROM orders
	WHERE quantity IS NOT NULL
	GROUP BY market,region,product_id
)
SELECT 
	product_id,
	discount,
	market,
	region,
	sales,
	quantity,
	sales/(unit_price *(1-discount)) AS calculated_quantity
FROM orders
INNER JOIN unit_prices
USING(market,region,product_id)
WHERE quantity IS NULL;

product_id  discount  market  ...    sales  quantity  calculated_quantity
0  TEC-STA-10003330      0.00  Africa  ...  506.640       NaN                  2.0
1  FUR-ADV-10000571      0.00    EMEA  ...  438.960       NaN                  4.0
2   FUR-BO-10001337      0.15      US  ...  308.499       NaN                  3.0
3  TEC-STA-10004542      0.00  Africa  ...  160.320       NaN                  4.0
4  FUR-ADV-10004395      0.00    EMEA  ...   84.120       NaN                  2.0

[5 rows x 7 columns]